In [ ]:
from main import*
from functools import partial
import time as tm
m = bi(platform='cpu')

g:\OneDrive\Travail\Max Planck\Projects\BI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


jax.local_device_count 16


In [ ]:
from numpyro import handlers
with handlers.seed(rng_seed=0):  # random.PRNGKey(0) is used

    x = numpyro.sample('x', dist.Beta(1, 1))    # uses a PRNGKey split from random.PRNGKey(0)

    y = numpyro.sample('y', dist.Bernoulli(x))  # uses different PRNGKey split from the last one

# Probability distributions

In [ ]:
import inspect
import numpyro as numpyro

# Get all names defined in numpyro.distributions
all_names = dir(numpyro.distributions)

# Create a dictionary with all names
class_dict = {name: getattr(numpyro.distributions, name) for name in all_names}

# Create a Python file and write the import statement and class with methods to it
with open("dists.py", "w") as file:
    # Write the import statement
    file.write("import numpyro as numpyro\n\n")
    
    # Write the class definition with __init__ method
    file.write("class Dist:\n\n")
    file.write("    def __init__(self):\n")
    file.write("        pass\n\n")
    
    # Write the generated methods with enhanced docstrings and dynamic signatures
    for key, value in class_dict.items():
        if callable(value):
            try:
                # Use inspect to get the signature of the function
                signature = inspect.signature(value)
                parameters = signature.parameters
                
                # Build the method signature string
                param_str = ", ".join([str(param) for param in parameters.values()])
                full_signature = f"{param_str}, sample_shape=()"
                
                # Create the method definition string with dynamic arguments
                method_name = key.lower()
                method_str = f"    def {method_name}(self, name, {full_signature}):\n"
                
                # Create a docstring with the method name and parameters
                docstring = f"{value.__name__} distribution.\n\n"
                docstring += "    Arguments:\n"
                for param in parameters.values():
                    docstring += f"        {param}\n"
                docstring += "        sample_shape: Shape of samples to be drawn.\n"
                
                # Format and indent the docstring
                indented_docstring = '\n    '.join(docstring.splitlines())
                method_str += f'        """\n        {indented_docstring}\n        """\n'
                
                # Create the argument string for the return statement
                arg_names = [param.name for param in parameters.values()]
                arg_str = ", ".join([f"{arg}={arg}" for arg in arg_names])
                
                # Add the method body with explicit argument passing
                method_str += f"        return numpyro.sample(name, numpyro.distributions.{value.__name__}({arg_str}).expand(sample_shape))\n"
                
                # Write the method string to the file
                file.write(method_str + "\n")
            except Exception as e:
                print(f"Error creating method for {key}: {e}")
        else:
            print(f"Ignoring non-callable object for key {key}: {value}")


Ignoring non-callable object for key __all__: ['biject_to', 'constraints', 'kl_divergence', 'transforms', 'AsymmetricLaplace', 'AsymmetricLaplaceQuantile', 'Bernoulli', 'BernoulliLogits', 'BernoulliProbs', 'Beta', 'BetaBinomial', 'BetaProportion', 'Binomial', 'BinomialLogits', 'BinomialProbs', 'Categorical', 'CategoricalLogits', 'CategoricalProbs', 'Cauchy', 'Chi2', 'Delta', 'Dirichlet', 'DirichletMultinomial', 'DiscreteUniform', 'Distribution', 'EulerMaruyama', 'Exponential', 'ExpandedDistribution', 'FoldedDistribution', 'Gamma', 'GammaPoisson', 'GaussianCopula', 'GaussianCopulaBeta', 'GaussianRandomWalk', 'Geometric', 'GeometricLogits', 'GeometricProbs', 'Gompertz', 'Gumbel', 'HalfCauchy', 'HalfNormal', 'ImproperUniform', 'Independent', 'InverseGamma', 'Kumaraswamy', 'LKJ', 'LKJCholesky', 'Laplace', 'LeftTruncatedDistribution', 'Logistic', 'LogNormal', 'LogUniform', 'MatrixNormal', 'MaskedDistribution', 'Mixture', 'MixtureSameFamily', 'MixtureGeneral', 'Multinomial', 'MultinomialLogi

In [ ]:
from dists import Dist
dist = Dist()
dist.exponential

<bound method Dist.exponential of <dists.Dist object at 0x0000015E632C1430>>

# Sampler for pdf

In [ ]:
import inspect
import numpyro as numpyro

# Get all names defined in numpyro.distributions
all_names = dir(numpyro.distributions)

# Create a dictionary with all names
class_dict = {name: getattr(numpyro.distributions, name) for name in all_names}

# Create a Python file and write the import statement and class with methods to it
with open("samplers.py", "w") as file:
    # Write the import statement
    file.write("import numpyro as numpyro\n")
    file.write("from numpyro import handlers\n\n")
    
    # Write the class definition with __init__ method
    file.write("class sampler:\n\n")
    file.write("    def __init__(self):\n")
    file.write("        pass\n\n")
    
    # Write the generated methods with enhanced docstrings and dynamic signatures
    for key, value in class_dict.items():
        if callable(value):
            try:
                # Use inspect to get the signature of the function
                signature = inspect.signature(value)
                parameters = signature.parameters
                
                # Build the method signature string
                param_str = ", ".join([str(param) for param in parameters.values()])
                full_signature = f"{param_str}, sample_shape=[1]"
                
                # Create the method definition string with dynamic arguments
                method_name = key.lower()
                method_str = f"    def {method_name}(self, {full_signature}, seed=0):\n"
                
                # Create a docstring with the method name and parameters
                docstring = f"{value.__name__} distribution.\n\n"
                docstring += "        Arguments:\n"
                for param in parameters.values():
                    docstring += f"        {param}\n"
                docstring += "        sample_shape: Shape of samples to be drawn.\n"
                
                # Format and indent the docstring
                indented_docstring = '\n    '.join(docstring.splitlines())
                method_str += f'        """\n        {indented_docstring}\n        """\n'
                
                # Create the argument string for the return statement
                arg_names = [param.name for param in parameters.values()]
                arg_str = ", ".join([f"{arg}={arg}" for arg in arg_names])

                # Add the method body with explicit argument passing
                method_str += f"        with handlers.seed(rng_seed=seed):\n"
                
                # Build the distribution call with sample_shape
                method_str += f"            distribution = numpyro.distributions.{value.__name__}({arg_str})\n"
                method_str += f"            return numpyro.sample('x', distribution.expand(sample_shape))\n"
                
                
                # Write the method string to the file
                file.write(method_str + "\n")
            except Exception as e:
                print(f"Error creating method for {key}: {e}")
        else:
            print(f"Ignoring non-callable object for key {key}: {value}")


Ignoring non-callable object for key __all__: ['biject_to', 'constraints', 'kl_divergence', 'transforms', 'AsymmetricLaplace', 'AsymmetricLaplaceQuantile', 'Bernoulli', 'BernoulliLogits', 'BernoulliProbs', 'Beta', 'BetaBinomial', 'BetaProportion', 'Binomial', 'BinomialLogits', 'BinomialProbs', 'Categorical', 'CategoricalLogits', 'CategoricalProbs', 'Cauchy', 'Chi2', 'Delta', 'Dirichlet', 'DirichletMultinomial', 'DiscreteUniform', 'Distribution', 'EulerMaruyama', 'Exponential', 'ExpandedDistribution', 'FoldedDistribution', 'Gamma', 'GammaPoisson', 'GaussianCopula', 'GaussianCopulaBeta', 'GaussianRandomWalk', 'Geometric', 'GeometricLogits', 'GeometricProbs', 'Gompertz', 'Gumbel', 'HalfCauchy', 'HalfNormal', 'ImproperUniform', 'Independent', 'InverseGamma', 'Kumaraswamy', 'LKJ', 'LKJCholesky', 'Laplace', 'LeftTruncatedDistribution', 'Logistic', 'LogNormal', 'LogUniform', 'MatrixNormal', 'MaskedDistribution', 'Mixture', 'MixtureSameFamily', 'MixtureGeneral', 'Multinomial', 'MultinomialLogi

In [ ]:
from sampler import sample
sample = sample()
sample.exponential(sample_shape = [2,2], rng_seed = 2)

Array([[0.23293737, 0.15043552],
       [1.0386753 , 0.05323007]], dtype=float32)

# Jax functions

In [ ]:
init_key, sample_key = random.split(random.PRNGKey(int(r.randint(0, 10000000))))
init_key = jnp.array(init_key)

@partial(jit, static_argnums=(1, 2,))
def vec_to_mat(arr, N, K):
    return jnp.reshape(arr, (N, K))

@jit
def jax_LinearOperatorDiag(s, cov):    
    def multiply_with_s(a):
        return jnp.multiply(a, s)
    vectorized_multiply = vmap(multiply_with_s)
    return jnp.transpose(vectorized_multiply(cov))
import jax.numpy as jnp

@jit
def diag_pre_multiply(v, m):
    return jnp.matmul(jnp.diag(v), m)

@jit
def random_centered(sigma, cor_mat, offset_mat):
    """Generate the centered matrix of random factors 

    Args:
        sigma (vector): Prior, vector of length N
        cor_mat (2D array): correlation matrix, cholesky_factor_corr of dim N, N
        offset_mat (2D array): matrix of offsets, matrix of dim N*k

    Returns:
        _type_: 2D array
    """
    return jnp.dot(diag_pre_multiply(sigma, cor_mat), offset_mat)



# Data manipulation

In [ ]:
import numpy as np
def index(df, cols = 'all'):
    index_map = {}
    if cols == 'all':
        colCat = list(df.select_dtypes(['object']).columns)    
        for a in range(len(colCat)):                
            df["index_"+ colCat[a]] =  df.loc[:,colCat[a]].astype("category").cat.codes
            df["index_"+ colCat[a]] = df["index_"+ colCat[a]].astype(np.int64)
            index_map[colCat[a]] = dict(enumerate(df[colCat[a]].astype("category").cat.categories ) )
    else:
        if isinstance(cols, list) == False:
            cols = [cols]
        for a in range(len(cols)):
            df["index_"+ cols[a]] =  df.loc[:,cols[a]].astype("category").cat.codes
            df["index_"+ cols[a]] = df["index_"+ cols[a]].astype(np.int64)
            index_map[cols[a]] = dict(enumerate(df[cols[a]].astype("category").cat.categories ) )
    df.columns = df.columns.str.replace('.', '_')
    df.columns = df.columns.str.replace(' ', '_')
    
    return df

# Rethinking

## 1. Continuous variable: Model (model 4.3)

In [3]:
import time as tm
from main import*
# setup platform------------------------------------------------
m = bi(platform='cpu')

# import data ------------------------------------------------
m.data('/home/sosa/BI/data/Howell1.csv', sep=';') 
m.data = m.data[m.data .age > 18]
m.data.weight = m.data.weight - m.data.weight.mean()
m.data.age = m.data.age - m.data.age.mean()
weight = jnp.array(m.data.weight.values)
height = jnp.array(m.data.height.values)
# TODO: use jax arrays with hugging face package

m.data = dict(height = height, weight = weight)

 # define model ------------------------------------------------
def model(height, weight):
    s = uniform('s', [1], 0, 50)
    a = normal('a', [1], 178, 20)
    b = normal('b', [1], 0, 1)  
    sample("y", Normal(a + b * weight, s), obs=height)

# Run sampler ------------------------------------------------
m.run(model) 
m.sampler.print_summary(0.89)


jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:00<00:00, 1000.53it/s, 7 steps of size 7.33e-01. acc. prob=0.92]

BI took: 1.0614 seconds

                mean       std    median      5.5%     94.5%     n_eff     r_hat
      a[0]    154.65      0.28    154.66    154.21    155.07    542.38      1.00
      b[0]      0.90      0.04      0.90      0.84      0.97    532.56      1.00
      s[0]      5.15      0.20      5.14      4.81      5.44    484.57      1.00

Number of divergences: 0


## 2. Categorical variable: Model (model 5.9)

In [ ]:
milk = pd.read_csv("../data/milk.csv", sep=";")
d = milk
d.clade.unique()
d["clade_id"] = d.clade.astype("category").cat.codes
def model(clade_id, K):
    a = numpyro.sample("a", dist.Normal(0, 0.5).expand([len(set(clade_id))]))
    sigma = numpyro.sample("sigma", dist.Exponential(1))
    mu = a[clade_id]
    numpyro.sample("height", dist.Normal(mu, sigma), obs=K)
                               


In [138]:
 # setup platform------------------------------------------------
m = bi(platform='cpu')
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/milk.csv', sep=';') 
m.data = index(m.data, cols = "clade")
m.data["K"] = m.data ["kcal_per_g"].pipe(lambda x: (x - x.mean()) / x.std())
index_clade = jnp.array(m.data.index_clade.values, dtype=jnp.int32)
K = jnp.array(m.data.K.values, dtype=jnp.float32)

m.data = dict(K = K, index_clade = index_clade)

 # define model ------------------------------------------------
def model(K, index_clade):
    s = exponential('s', [1], 1)
    a = normal('a', [1], 0, 0.5)
    m = a[index_clade]
    sample("y", Normal(m, s), obs=K)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)


jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:00<00:00, 1076.86it/s, 1023 steps of size 1.09e-03. acc. prob=0.87]

BI took: 1.0159 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
a[0],0.03,0.16,-0.22,0.28
s[0],1.00,0.12,0.84,1.22


## 3. Continuous interactions terms (model 8.3)

In [8]:
 # setup platform------------------------------------------------
m = bi(platform='cpu')
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/tulips.csv', sep=';') 
m.data["blooms_std"] = m.data.blooms /m.data.blooms.max()
m.data["water_cent"] = m.data.water - m.data.water.mean()
m.data["shade_cent"] = m.data.shade - m.data.shade.mean()

water_cent = jnp.array(m.data.water_cent.values)
blooms_std = jnp.array(m.data.blooms_std.values)
shade_cent = jnp.array(m.data.shade_cent.values)


m.data = dict(water_cent = water_cent, blooms_std = blooms_std, shade_cent = shade_cent)

 # define model ------------------------------------------------
def model(blooms_std,water_cent, shade_cent):
    sigma = exponential('sigma', [1], 1)
    bws = normal('bws', [1], 0, 0.25)
    bs = normal('bs', [1], 0, 0.25)
    bw = normal('bw', [1], 0, 0.25)
    a = normal('a', [1], 0.5, 0.25)
    mu = a + bw*water_cent + bs*shade_cent + bws*water_cent*shade_cent
    sample("y", Normal(mu, sigma), obs=blooms_std)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)


jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:00<00:00, 1191.35it/s, 3 steps of size 7.12e-01. acc. prob=0.88]

BI took: 0.9271 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
a[0],0.36,0.03,0.32,0.40
bs[0],-0.11,0.03,-0.16,-0.06
bw[0],0.20,0.04,0.15,0.27
bws[0],-0.14,0.04,-0.20,-0.07
sigma[0],0.14,0.02,0.11,0.17


## 4. Binomial (model 11.1)

In [119]:
import pandas as pd
d = pd.read_csv('/home/sosa/BI/data/chimpanzees.csv', sep = ';')
d["treatment"] = 1 + d.prosoc_left + 2 * d.condition
d["side"] = d.prosoc_left  # right 0, left 1
d["cond"] = d.condition  # no partner 0, partner 1

In [123]:
m = bi()
m.data = dict(pulled_left = jnp.array(d.pulled_left.values, dtype=jnp.int32))
def model(pulled_left):
    a = normal('a', [1], 0, 10)
    sample("y", Binomial(logits=a), obs=pulled_left)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model, init_strategy = numpyro.infer.initialization.init_to_mean()) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)


jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:00<00:00, 1749.34it/s, 1 steps of size 9.55e-01. acc. prob=0.93]

BI took: 0.6800 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
a[0],0.33,0.08,0.19,0.47


##  5. Binomial with indices (model 11.4)

In [29]:
 # setup platform------------------------------------------------
m = bi(platform='cpu')
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/chimpanzees.csv', sep=';') 
m.data["treatment"] = 1 + m.data.prosoc_left + 2 * m.data.condition
treatment = jnp.array(m.data["treatment"], dtype=jnp.int32)
actor = jnp.array(m.data["actor"] )
n_actor = len(jnp.unique(actor))
n_treatment= len(jnp.unique(treatment))
n_actor

jax.local_device_count 32


7

In [33]:
 # setup platform------------------------------------------------
m = bi(platform='cpu')
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/chimpanzees.csv', sep=';') 
m.data["treatment"] = 1 + m.data.prosoc_left + 2 * m.data.condition
treatment = jnp.array(m.data["treatment"], dtype=jnp.int32)
actor = jnp.array(m.data["actor"] )
pulled_left = jnp.array(m.data["pulled_left"] )
n_actor = len(jnp.unique(actor))
n_treatment= len(jnp.unique(treatment))

m.data = dict(
    actor = actor,
    treatment = treatment,
    pulled_left = pulled_left,
    n_actor = n_actor,
    n_treatment = n_treatment
)

def model(n_actor, n_treatment, actor, treatment, pulled_left):
    a = normal('a', [n_actor], 0, 1.5)
    b = normal('b', [n_treatment], 0, 0.5)
    p = a[actor] + b[treatment]
    sample("y", Binomial(1, logits=p), obs=pulled_left)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:03<00:00, 281.56it/s, 1023 steps of size 6.48e-04. acc. prob=0.80]


BI took: 3.6782 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
a[0],0.46,1.30,-1.82,2.12
a[1],0.07,0.26,-0.31,0.52
a[2],3.73,0.45,3.02,4.40
a[3],-0.11,0.20,-0.41,0.22
a[4],-0.01,0.21,-0.32,0.31
a[5],0.10,0.15,-0.15,0.33
a[6],1.69,0.18,1.41,1.99
b[0],0.22,0.46,-0.44,0.97
b[1],-0.62,0.24,-0.97,-0.22
b[2],-0.08,0.24,-0.48,0.23


## 6. Poisson (model 11.10) PB

In [110]:
import tensorflow as tf
import math
# setup platform------------------------------------------------
m = bi()
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/Kline.csv', sep=';') 
m.data["P"] = m.data.population.apply(math.log).pipe(lambda x: (x - x.mean()) / x.std())
m.data["contact_id"] = (m.data.contact == "high").astype(int)
m.data = dict(total_tools=m.data.total_tools.values, P=m.data.P.values, cid=m.data.contact_id.values)
def model(cid, P, total_tools):
    a = normal('a', [1], 3, 0.5)
    b = normal('b', [1], 0, 0.2)
    l = jnp.exp(a[cid] + b[cid]*P)
    sample("y", Poisson(l), obs=total_tools)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:00<00:00, 1156.54it/s, 511 steps of size 1.04e-03. acc. prob=0.85]


BI took: 0.9525 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
a[0],3.49,0.06,3.40,3.57
b[0],0.34,0.05,0.26,0.41


## 7. Negative binomial (model 11.12) (PB estimation)

In [53]:
import tensorflow_probability.substrates.jax.distributions as tfd
init_key, sample_key = random.split(random.PRNGKey(int(r.randint(0, 10000000))))
init_key = jnp.array(init_key)
num_days = 30
y = tfd.Poisson(rate=1.5).sample(seed = init_key, sample_shape=(num_days,))
num_weeks = 4
y_new = tfd.Poisson(rate=0.5 * 7).sample(seed = init_key, sample_shape=(num_weeks,))
y_all = np.concatenate([y, y_new])
exposure = np.concatenate([np.repeat(1, 30), np.repeat(7, 4)])
monastery = np.concatenate([np.repeat(0, 30), np.repeat(1, 4)])
d = pd.DataFrame.from_dict(dict(y=y_all, days=exposure, monastery=monastery))
d["log_days"] = d.days.pipe(np.log)

/home/sosa/.local/lib/python3.10/site-packages/jax/_src/numpy/array_methods.py:66: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return lax_numpy.astype(arr, dtype)
/home/sosa/.local/lib/python3.10/site-packages/tensorflow_probability/python/internal/backend/jax/ops.py:339: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return np.array(value, dtype=dtype)
/home/sosa/.local/lib/python3.10/site-packages/tensorflow_probability/python/internal/

In [61]:
d

,y,days,monastery,log_days
0,1.0,1,0,0.00000
1,2.0,1,0,0.00000
2,1.0,1,0,0.00000
3,1.0,1,0,0.00000
4,2.0,1,0,0.00000
5,3.0,1,0,0.00000
6,3.0,1,0,0.00000
7,2.0,1,0,0.00000
8,2.0,1,0,0.00000
9,2.0,1,0,0.00000


In [63]:
# setup platform------------------------------------------------
m = bi()
# import data ------------------------------------------------
m.data = dict(
    log_days = jnp.array(d.log_days.values),
    monastery = jnp.array(d.monastery.values),
    output = jnp.array(d.y.values)
)

def model(log_days, monastery, output):
    a = normal('a', [1], 0, 1)
    b = normal('b', [1], 0, 1)
    l = log_days + a +  b * monastery
    sample("y", Poisson(rate = l), obs=output)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


  0%|          | 0/1000 [00:00<?, ?it/s]

sample: 100%|██████████| 1000/1000 [00:00<00:00, 1516.49it/s, 1 steps of size 7.71e-01. acc. prob=0.92]

BI took: 0.7408 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
a[0],1.43,0.21,1.07,1.72
b[0],0.28,0.66,-0.80,1.34


## 8. Multinomial (model 11.13)

In [65]:
# simulate career choices among 500 individuals
N = 500  # number of individuals
income = np.array([1, 2, 5])  # expected income of each career
score = 0.5 * income  # scores for each career, based on income

# next line converts scores to probabilities
p = jnp.array(tf.nn.softmax(score))

# now simulate choice
# outcome career holds event type values, not counts
career = tfd.Categorical(probs=p).sample(seed = init_key, sample_shape = N)
result = [income[index] for index in career]
data = {'career': career, 'income': result}
d = pd.DataFrame(data)
career = jnp.array(d.career.values)
career_income = jnp.array(d.income.values)
income = jnp.array(income)
m.data = dict(
    income = income,
    career = career
)

In [68]:
def model(income, career):
    a = normal('a', [2], 0, 1)
    b = halfnormal('b', [1], 0.5)
    s_1 = a[0] + b * income[0]
    s_2 = a[1] + b * income[1]
    s_3 = a[0] + b * income[0]
    p = jax.nn.softmax(jnp.stack([s_1[0], s_2[0], s_3[0]]))
    sample("y", Categorical(probs =  p[career]), obs=career)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

sample: 100%|██████████| 1000/1000 [00:00<00:00, 1015.24it/s, 11 steps of size 1.04e-01. acc. prob=0.93]


BI took: 1.0875 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
a[0],0.54,0.70,-0.50,1.61
a[1],-0.56,0.71,-1.65,0.66
b[0],0.34,0.27,0.00,0.71


## 9. Beta binomial (model m12.1)

In [75]:
# setup platform------------------------------------------------
m = bi()
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/UCBadmit.csv', sep=';') 
m.data["gid"] = (m.data["applicant.gender"] != "male").astype(int)
gid = jnp.array(m.data["gid"].astype('int32').values)
applications = jnp.array(m.data["applications"].astype('float32').values)
admit = jnp.array(m.data["admit"].astype('float32').values)

m.data = dict(
    gid = gid,
    applications = applications,
    admit =  admit
)

def model(gid, applications, admit):
    phi = exponential('phi', [1], 1)
    alpha = normal('alpha', [2], 0., 1.5)
    theta = phi + 2
    pbar = jax.nn.sigmoid(alpha[gid])
    concentration1 = pbar*theta
    concentration0 = (1 - pbar) * theta
    sample("y", BetaBinomial(total_count = applications, concentration1 = concentration1, concentration0 = concentration0), obs=admit)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:01<00:00, 804.99it/s, 3 steps of size 6.20e-01. acc. prob=0.86]


BI took: 1.4778 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
alpha[0],-0.40,0.43,-1.09,0.30
alpha[1],-0.34,0.43,-0.96,0.37
phi[0],0.98,0.75,0.00,1.96


## 10. Negative-binomial

In [104]:
# setup platform------------------------------------------------
m = bi()
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/UCBadmit.csv', sep=';') 
m.data["gid"] = (m.data["applicant.gender"] != "male").astype(int)
gid = jnp.array(m.data["gid"].astype('int32').values)
applications = jnp.array(m.data["applications"].astype('float32').values)
admit = jnp.array(m.data["admit"].astype('float32').values)

m.data = dict(
    gid = gid,
    applications = applications,
    admit =  admit
)

def model(gid, applications, admit):
    phi = exponential('phi', [1], 1)
    alpha = normal('alpha', [2], 0., 1.5)
    theta = phi + 2
    pbar = jax.nn.sigmoid(alpha[gid])
    concentration1 = pbar*theta
    concentration0 = (1 - pbar) * theta
    sample("y", BetaBinomial(total_count = applications, concentration1 = concentration1, concentration0 = concentration0), obs=admit)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:01<00:00, 906.62it/s, 3 steps of size 6.20e-01. acc. prob=0.86]


BI took: 1.3320 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
alpha[0],-0.40,0.43,-1.09,0.30
alpha[1],-0.34,0.43,-0.96,0.37
phi[0],0.98,0.75,0.00,1.96


## 11. Zero inflated outcomes

In [139]:
from jax.scipy.special import expit
r.seed(42)
# Define parameters
prob_drink = 0.2  # 20% of days
rate_work = 1     # average 1 manuscript per day

# sample one year of production
N = 365

np.random.seed(365)
drink = np.random.binomial(1, prob_drink, N)
y = (1 - drink) * np.random.poisson(rate_work, N)

# setup platform------------------------------------------------
m = bi()
# import data ------------------------------------------------

m.data = dict(
    y = jnp.array(y)
)

def model(y):
    al = normal('al', [1], 1, 0.5)
    ap = normal('ap', [1], -1.5, 1)
    p = expit(ap)
    lambda_ = jnp.exp(al)
    sample("y", ZeroInflatedPoisson(p, lambda_), obs=y)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:00<00:00, 1446.23it/s, 1 steps of size 5.90e-01. acc. prob=0.93]

BI took: 0.7985 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
al[0],0.11,0.08,-0.0,0.25
ap[0],-1.37,0.33,-1.9,-0.85


## 12. OrderedLogistic

In [125]:
import numpyro.distributions as dist
# setup platform------------------------------------------------
m = bi()
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/Trolley.csv', sep=';') 
d = m.data
# discrete proportion of each response value
pr_k = d.response.value_counts().sort_index().values / d.shape[0]
# cumsum converts to cumulative proportions
cum_pr_k = jnp.cumsum(pr_k, -1)
logit = lambda x: jnp.log(x / (1 - x))  # convenience function
lco = logit(cum_pr_k)

jax.local_device_count 32


In [129]:
import numpyro.distributions as dist
from numpyro.distributions.transforms import OrderedTransform
# setup platform------------------------------------------------
m = bi()
m.data = dict(response = jnp.array(d.response.values - 1))
def model(response):
    cutpoints = numpyro.sample(
        "cutpoints",
        dist.TransformedDistribution(
            dist.Normal(0, 1.5).expand([6]), OrderedTransform()
        ),
    )
    numpyro.sample("R", dist.OrderedLogistic(0, cutpoints), obs=response)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:01<00:00, 914.70it/s, 7 steps of size 4.63e-01. acc. prob=0.90] 


BI took: 1.2182 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
cutpoints[0],-1.92,0.03,-1.97,-1.87
cutpoints[1],-1.27,0.02,-1.30,-1.23
cutpoints[2],-0.72,0.02,-0.75,-0.69
cutpoints[3],0.25,0.02,0.22,0.28
cutpoints[4],0.89,0.02,0.86,0.93
cutpoints[5],1.77,0.03,1.73,1.82


## 13. Varying interceps

In [85]:
# setup platform------------------------------------------------
m = bi()
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/reedfrogs.csv', sep=';') 
m.data["tank"] = np.arange(m.data.shape[0])
tank = jnp.array(m.data["tank"].astype('int32').values)
density = jnp.array(m.data["density"].astype('float32').values)
surv = jnp.array(m.data["surv"].astype('int32').values)
m.data = dict(
    tank = tank,
    surv = surv
)

def model(tank, surv):
    sigma = exponential('sigma', [1], 1)
    a_bar = normal('a_bar', [1], 0., 1.5)
    alpha = normal('alpha', [48], a_bar, sigma)
    p = jnp.squeeze(alpha[tank])[0]
    sample("y", Binomial(total_count = density, logits = p), obs=surv)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:00<00:00, 1251.10it/s, 15 steps of size 1.06e-01. acc. prob=0.77]


BI took: 0.9192 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
a_bar[0],0.83,0.90,-0.12,2.42
alpha[0],0.83,0.07,0.72,0.92
alpha[1],0.80,1.55,-1.53,3.38
alpha[2],0.82,1.86,-1.22,3.84
alpha[3],0.83,1.53,-1.36,3.15
alpha[4],0.84,1.78,-2.14,3.17
alpha[5],0.87,1.54,-1.71,3.08
alpha[6],0.77,1.78,-1.66,3.43
alpha[7],0.90,1.52,-1.68,2.92
alpha[8],0.79,1.57,-1.19,3.30


## 13. Varying effects

In [6]:
import numpyro.distributions as dist
a = 3.5  # average morning wait time
b = -1  # average difference afternoon wait time
sigma_a = 1  # std dev in intercepts
sigma_b = 0.5  # std dev in slopes
rho = -0.7  # correlation between intercepts and slopes
Mu = jnp.array([a, b])
cov_ab = sigma_a * sigma_b * rho
Sigma = jnp.array([[sigma_a**2, cov_ab], [cov_ab, sigma_b**2]])
jnp.array([1, 2, 3, 4]).reshape(2, 2).T
sigmas = jnp.array([sigma_a, sigma_b])  # standard deviations
Rho = jnp.array([[1, rho], [rho, 1]])  # correlation matrix

# now matrix multiply to get covariance matrix
Sigma = jnp.diag(sigmas) @ Rho @ jnp.diag(sigmas)

N_cafes = 2000
seed = random.PRNGKey(5)  # used to replicate example
vary_effects = dist.MultivariateNormal(Mu, Sigma).sample(seed, (N_cafes,))
a_cafe = vary_effects[:, 0]
b_cafe = vary_effects[:, 1]

seed = random.PRNGKey(22)
N_visits = 10
afternoon = jnp.tile(jnp.arange(2), N_visits * N_cafes // 2)
cafe_id = jnp.repeat(jnp.arange(N_cafes), N_visits)
mu = a_cafe[cafe_id] + b_cafe[cafe_id] * afternoon
sigma = 0.5  # std dev within cafes
wait = dist.Normal(mu, sigma).sample(seed)
d = pd.DataFrame(dict(cafe=cafe_id, afternoon=afternoon, wait=wait))

In [7]:
# import data ------------------------------------------------
m = bi()
m.data = dict(
    cafe = cafe_id, 
    wait = wait, 
    N_cafes = N_cafes
)
def model(cafe, wait, N_cafes):
    a = numpyro.sample("a", dist.Normal(5, 2))
    b = numpyro.sample("b", dist.Normal(-1, 0.5))
    sigma_cafe = exponential('sigma_cafe', [2], 1)
    sigma = exponential('sigma', [1], 1)
    Rho = numpyro.sample("Rho", dist.LKJ(2, 2))
    cov = jnp.outer(sigma_cafe, sigma_cafe) * Rho
    a_cafe_b_cafe = numpyro.sample(
        "a_cafe,b_cafe", dist.MultivariateNormal(jnp.stack([a, b]), cov).expand([N_cafes])
    )
    a_cafe, b_cafe = a_cafe_b_cafe[:, 0], a_cafe_b_cafe[:, 1]
    mu = a_cafe[cafe] + b_cafe[cafe] * afternoon
    sample("y", Normal(mu, sigma), obs=wait)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:01<00:00, 628.21it/s, 15 steps of size 3.00e-01. acc. prob=0.85]


BI took: 4.8678 seconds
BI took: 4.8680 seconds

                         mean       std    median      5.5%     94.5%     n_eff     r_hat
           Rho[0,0]      1.00      0.00      1.00      1.00      1.00       nan       nan
           Rho[0,1]     -0.72      0.15     -0.74     -0.93     -0.50    382.49      1.00
           Rho[1,0]     -0.72      0.15     -0.74     -0.93     -0.50    382.49      1.00
           Rho[1,1]      1.00      0.00      1.00      1.00      1.00     30.82      1.00
                  a      3.11      0.24      3.10      2.75      3.52    521.48      1.00
 a_cafe,b_cafe[0,0]      1.29      0.22      1.30      0.99      1.67    524.79      1.00
 a_cafe,b_cafe[0,1]     -0.07      0.27     -0.08     -0.54      0.30    333.80      1.00
 a_cafe,b_cafe[1,0]      2.61      0.21      2.60      2.28      2.96    462.74      1.00
 a_cafe,b_cafe[1,1]     -0.65      0.23     -0.65     -1.04     -0.32    402.36      1.00
 a_cafe,b_cafe[2,0]      2.57      0.25      2.57  

In [40]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
import numpy as np
#try:
#  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
#except:
#  pass

nest_asyncio.apply()
stan_code = """ 
data{
    int len;
    int N_cafes;
    vector[len] wait;
    array[len] int afternoon;
    array[len] int cafe;
}
parameters{
    vector[N_cafes] b_cafe;
    vector[N_cafes] a_cafe;
    real a;
    real b;
    vector<lower=0>[2] sigma_cafe;
    real<lower=0> sigma;
    corr_matrix[2] Rho;
}
model{
    vector[len] mu;
    Rho ~ lkj_corr( 2 );
    sigma ~ exponential( 1 );
    sigma_cafe ~ exponential( 1 );
    b ~ normal( -1 , 0.5 );    
    a ~ normal( 5 , 2 );
    {
        array[N_cafes] vector[2] YY;
        vector[2] MU;
        MU = [ a , b ]';
        for ( j in 1:N_cafes ) YY[j] = [ a_cafe[j] , b_cafe[j] ]';
        YY ~ multi_normal( MU , quad_form_diag(Rho , sigma_cafe) );
    }
    for ( i in 1:len ) {
        mu[i] = a_cafe[cafe[i]] + b_cafe[cafe[i]] * afternoon[i];        
    }
    
    wait ~ normal( mu , sigma );

}
"""
data = {
    'wait' : d['wait'].values.astype(float),
    'afternoon' : d['afternoon'].values.astype(int),
    'cafe' : d['cafe'].values.astype(int)+1,
    'N_cafes' : N_cafes,
    'len' : len(d['wait'].values)
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=1, num_samples=500, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_4314cm3u/model_4xoavcff.stan', line 20, column 4: It
    is suggested to reparameterize your model to replace lkj_corr with
    lkj_corr_cholesky, the Cholesky factor variant. lkj_corr tends to run
    slower, consume more memory, and has higher risk of numerical errors.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/1000)
Sampling:  10% (100/1000)
Sampling:  20% (200/1000)
Sampling:  30% (300/1000)
Sampling:  40% (400/1000)
Sampling:  50% (500/1000)
Sampling:  50% (501/1000)
Sampling:  60% (600/1000)
Sampling:  70% (700/1000)
Sampling:  80% (800/1000)
Sampling:  90% (900/1000)
Sampling: 100% (1000/1000)
Sampling: 100% (1000/1000), done.
Messages received during sampling:
  Gradient evaluation took 0.012288 

Pystan took: 1715.6098 seconds


## 14 Multiple random effects


## 15 Networks

## 15. STRABD example

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import jax.numpy as jnp
tfd = tfp.distributions
tfb = tfp.bijectors

# Make data
N_id = 100

# Covariates
Kinship = tfd.CholeskyLKJ(N_id, concentration=1.5).sample()
Dominant = np.ceil(tfd.CholeskyLKJ(N_id, concentration=1.5).sample())
Mass = tfd.Bernoulli(probs=0.4).sample(sample_shape=(N_id,))

# Organize into list
dyadic_preds = np.full((N_id, N_id, 3), np.nan)

dyadic_preds[:, :, 0] = Kinship
dyadic_preds[:, :, 1] = Dominant
dyadic_preds[:, :, 2] = Kinship * Dominant

# Set effect sizes
sr_mu = tf.constant([0, 0])
sr_sigma = tf.constant([2.2, 1.7])
sr_rho = 0.55
dr_mu = tf.constant([0, 0])
dr_sigma = 1.5
dr_rho = 0.6
sr_effects_1 = tf.constant([1.9, 1.3])
dr_effects_1 = tf.constant([1.2, 1.7, -2.2])


In [28]:
import json
import jax.numpy as jnp
with open('STRAND.json', 'r') as f:
    data = json.load(f)

In [29]:
data.keys()


dict_keys(['N_networktypes', 'N_id', 'N_responses', 'N_periods', 'N_individual_predictors', 'N_dyadic_predictors', 'outcomes', 'flows', 'individual_predictors', 'dyadic_predictors', 'N_block_predictors', 'N_groups_per_block_type', 'block_predictors', 'outcome_mode', 'exposure'])

In [4]:
data['N_networktypes']

1

In [5]:
data['N_id']

100

In [6]:
data['N_responses']

1

In [7]:
data['N_periods']

0

In [8]:
data['N_individual_predictors']

1

In [9]:
data['N_dyadic_predictors']

2

In [30]:
data['outcomes'] = jnp.array(data['outcomes']).reshape(data['N_id'],data['N_id'])
data['outcomes']

Array([[ 0,  0,  0, ...,  1,  9, 19],
       [ 1,  0,  0, ...,  0,  0,  2],
       [ 0,  0,  0, ...,  0,  3, 11],
       ...,
       [ 4,  0,  0, ...,  0,  1,  0],
       [ 3,  0,  0, ...,  0,  0,  9],
       [10,  6,  3, ...,  0, 11,  0]], dtype=int32)

In [18]:
data['flows']

0

In [31]:
data['individual_predictors'] = jnp.array(data['individual_predictors']['Mass'])
data['individual_predictors']

Array([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1], dtype=int32)

In [35]:
Kinship = jnp.array(data['dyadic_predictors']['Kinship']).reshape(data['N_id'],data['N_id'])
Kinship

Array([[ 1.        ,  0.08778227, -0.1032394 , ...,  0.08142215,
        -0.1223947 , -0.1078428 ],
       [ 0.08778227,  1.        , -0.1504998 , ...,  0.06893802,
        -0.1820744 , -0.00592948],
       [-0.1032394 , -0.1504998 ,  1.        , ..., -0.1213151 ,
         0.1459238 ,  0.1074177 ],
       ...,
       [ 0.08142215,  0.06893802, -0.1213151 , ...,  1.        ,
        -0.1211644 , -0.03577805],
       [-0.1223947 , -0.1820744 ,  0.1459238 , ..., -0.1211644 ,
         1.        ,  0.03175739],
       [-0.1078428 , -0.00592948,  0.1074177 , ..., -0.03577805,
         0.03175739,  1.        ]], dtype=float32)

In [36]:
Dominant = jnp.array(data['dyadic_predictors']['Dominant']).reshape(data['N_id'],data['N_id'])
Dominant

Array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int32)

In [37]:
data['N_block_predictors']

0

In [38]:
data['N_groups_per_block_type']

1

In [41]:
data['block_predictors'] = jnp.array(data['block_predictors'])
data['block_predictors'] 

In [42]:
data['outcome_mode'] 

2

In [44]:
jnp.array(data['exposure']).reshape(data['N_id'],data['N_id'])

Array([[14, 16, 16, ...,  8, 19, 21],
       [13, 11, 16, ..., 15, 21, 26],
       [14, 17, 13, ..., 17, 18, 14],
       ...,
       [22, 18, 15, ..., 14,  6, 13],
       [15, 17, 11, ...,  6, 15, 20],
       [10, 14, 19, ..., 16, 14, 15]], dtype=int32)

## 15.2 Rethinking example

In [4]:
import time as tm
from main import*
# setup platform------------------------------------------------
m = bi(platform='cpu')
kl_dyads  = pd.read_csv('/home/sosa/BI/data/kl_dyads')
d2 = pd.read_csv('/home/sosa/BI/data/kl_households'"", index_col=0)
kl_data = dict(
    N=kl_dyads.shape[0],
    N_households=kl_dyads.hidB.max(),
    did=kl_dyads.did.values - 1,
    hidA=kl_dyads.hidA.values - 1,
    hidB=kl_dyads.hidB.values - 1,
    giftsAB=kl_dyads.giftsAB.values,
    giftsBA=kl_dyads.giftsBA.values,
)
m.data = kl_data

def model(N_households, N, did, hidA, hidB, giftsAB, giftsBA, link=False):
    # gr matrix of varying effects
    Rho_gr = lkj("Rho_gr", [], 2, 4)
    sigma_gr = exponential("sigma_gr", [2], 1)
    cov = jnp.outer(sigma_gr, sigma_gr) * Rho_gr
    gr = multivariatenormal("gr", [N_households], 0, cov)

    # dyad effects
    z = normal("z",[2, N], 0, 1)
    L_Rho_d = lkjcholesky("L_Rho_d",[], 2, 8)
    sigma_d = exponential("sigma_d",[1], 1)
    d = numpyro.deterministic(
        "d", ((jnp.repeat(sigma_d, 2)[..., None] * L_Rho_d) @ z).T
    )

    a = normal('a', [1], 0, 1)
    lambdaAB = jnp.exp(a + gr[hidA, 0] + gr[hidB, 1] + d[did, 0])
    lambdaBA = jnp.exp(a + gr[hidB, 0] + gr[hidA, 1] + d[did, 1])
    sample("giftsAB", Poisson(lambdaAB), obs=giftsAB)
    sample("giftsBA", Poisson(lambdaBA), obs=giftsBA)

    # compute correlation matrix for dyads
    if link:
        numpyro.deterministic("Rho_d", L_Rho_d @ L_Rho_d.T)

# Run sampler ------------------------------------------------

m.run(model) 
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:04<00:00, 244.77it/s, 63 steps of size 9.02e-02. acc. prob=0.89]


BI took: 8.1093 seconds

                  mean       std    median      5.5%     94.5%     n_eff     r_hat
L_Rho_d[0,0]      1.00      0.00      1.00      1.00      1.00       nan       nan
L_Rho_d[0,1]      0.00      0.00      0.00      0.00      0.00       nan       nan
L_Rho_d[1,0]      0.88      0.04      0.88      0.82      0.93     53.74      1.02
L_Rho_d[1,1]      0.47      0.07      0.48      0.36      0.57     54.94      1.02
 Rho_gr[0,0]      1.00      0.00      1.00      1.00      1.00       nan       nan
 Rho_gr[0,1]     -0.39      0.20     -0.41     -0.71     -0.08    203.92      1.00
 Rho_gr[1,0]     -0.39      0.20     -0.41     -0.71     -0.08    203.92      1.00
 Rho_gr[1,1]      1.00      0.00      1.00      1.00      1.00    435.00      1.00
        a[0]      0.53      0.17      0.52      0.27      0.79    153.88      1.00
     gr[0,0]     -0.49      0.28     -0.48     -0.91     -0.03    252.20      1.01
     gr[0,1]      0.21      0.22      0.21     -0.15      0.55

In [ ]:
import numpy as onp
import numpyro as numpyro
import numpyro.distributions as dist
from numpyro.diagnostics import effective_sample_size, print_summary
from numpyro.infer import MCMC, NUTS, Predictive
def model(N_households, N, did, hidA, hidB, giftsAB, giftsBA, link=False):
    # gr matrix of varying effects
    Rho_gr = numpyro.sample("Rho_gr", dist.LKJ(2, 4))
    sigma_gr = numpyro.sample("sigma_gr", dist.Exponential(1).expand([2]))
    cov = jnp.outer(sigma_gr, sigma_gr) * Rho_gr
    gr = numpyro.sample("gr", dist.MultivariateNormal(0, cov).expand([N_households]))

    # dyad effects
    z = numpyro.sample("z", dist.Normal(0, 1).expand([2, N]))
    L_Rho_d = numpyro.sample("L_Rho_d", dist.LKJCholesky(2, 8))
    sigma_d = numpyro.sample("sigma_d", dist.Exponential(1))
    d = numpyro.deterministic(
        "d", ((jnp.repeat(sigma_d, 2)[..., None] * L_Rho_d) @ z).T
    )

    a = numpyro.sample("a", dist.Normal(0, 1))
    lambdaAB = jnp.exp(a + gr[hidA, 0] + gr[hidB, 1] + d[did, 0])
    lambdaBA = jnp.exp(a + gr[hidB, 0] + gr[hidA, 1] + d[did, 1])
    numpyro.sample("giftsAB", dist.Poisson(lambdaAB), obs=giftsAB)
    numpyro.sample("giftsBA", dist.Poisson(lambdaBA), obs=giftsBA)

    # compute correlation matrix for dyads
    if link:
        numpyro.deterministic("Rho_d", L_Rho_d @ L_Rho_d.T)


m14_7 = MCMC(NUTS(model), num_warmup=1000, num_samples=1000, num_chains=4)
m14_7.run(random.PRNGKey(0), **kl_data)

## 16. Gaussian Processes

In [6]:
import time as tm
from main import*
Kline2 = pd.read_csv('/home/sosa/BI/data/Kline2.csv', sep=";")
islandsDistMatrix = pd.read_csv('/home/sosa/BI/data/islandsDistMatrix.csv'"", index_col=0)
d = Kline2
d["society"] = range(1, 11)  # index observations

dat_list = dict(
    T=d.total_tools.values,
    P=d.population.values,
    society=d.society.values - 1,
    Dmat=islandsDistMatrix.values,
)

# setup platform------------------------------------------------
m14_8 = bi(platform='cpu')
m14_8.data = dat_list

@jit
def cov_GPL2(x, sq_eta, sq_rho, sq_sigma):
    N = x.shape[0]
    K = sq_eta * jnp.exp(-sq_rho * jnp.square(x))
    K = K.at[jnp.diag_indices(N)].add(sq_sigma)
    return K


def model(Dmat, P, society, T):
    a = exponential('a', [1], 1)
    b = exponential('b',[1],1)
    g = exponential('g',[1],1)
    etasq = exponential('etasq',[1],2)
    rhosq = exponential('rhosq',[1],0.5)

    # non-centered Gaussian Process prior
    SIGMA = cov_GPL2(Dmat, etasq, rhosq, 0.01)
    L_SIGMA = jnp.linalg.cholesky(SIGMA)
    z = normal('z', [10], 0, 1)
    k = (L_SIGMA @ z[..., None])[..., 0]
    lambda_ = a * P**b / g * jnp.exp(k[society])
    sample("T", Poisson(lambda_), obs=T)

# Run sampler ------------------------------------------------
start = tm.time()    
m14_8.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")
m14_8.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:01<00:00, 768.14it/s, 127 steps of size 3.22e-02. acc. prob=0.95]


BI took: 1.9520 seconds

                mean       std    median      5.5%     94.5%     n_eff     r_hat
      a[0]      1.33      0.90      1.11      0.15      2.60    394.53      1.02
      b[0]      0.28      0.09      0.28      0.13      0.41    209.03      1.00
  etasq[0]      0.20      0.20      0.14      0.01      0.42    280.91      1.00
      g[0]      0.59      0.59      0.43      0.02      1.23    302.88      1.02
  rhosq[0]      1.38      1.72      0.75      0.02      3.37    329.98      1.00
      z[0]     -0.49      0.79     -0.46     -1.53      0.84    241.76      1.00
      z[1]      0.37      0.76      0.39     -0.86      1.49    502.49      1.00
      z[2]     -0.26      0.76     -0.21     -1.73      0.68    636.05      1.00
      z[3]      0.93      0.65      0.95     -0.12      1.84    255.45      1.00
      z[4]      0.30      0.61      0.28     -0.52      1.37    330.08      1.00
      z[5]     -1.09      0.72     -1.02     -2.20     -0.05    271.82      1.00
   

In [ ]:

# import data ------------------------------------------------
m.data('/home/sosa/BI/data/Howell1.csv', sep=';') 
m.data = m.data[m.data .age > 18]
m.data.weight = m.data.weight - m.data.weight.mean()
m.data.age = m.data.age - m.data.age.mean()
weight = jnp.array(m.data.weight.values)
height = jnp.array(m.data.height.values)
# TODO: use jax arrays with hugging face package
# TODO: Add scale function
m.data = dict(height = height, weight = weight)

 # define model ------------------------------------------------
def model(height, weight):
    s = uniform('s', [1], 0, 50)
    a = normal('a', [1], 178, 20)
    b = normal('b', [1], 0, 1)  
    sample("y", Normal(a + b * weight, s), obs=height)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.summary()



# STRAND

In [ ]:
import json
import jax.numpy as jnp
with open('OUTPUT STRAND.json.json', 'r') as f:
    data = json.load(f)
data.keys()

dict_keys(['N_networktypes', 'N_id', 'N_responses', 'N_periods', 'N_individual_predictors', 'N_dyadic_predictors', 'outcomes', 'flows', 'individual_predictors', 'dyadic_predictors', 'N_block_predictors', 'N_groups_per_block_type', 'block_predictors', 'outcome_mode', 'exposure'])

In [ ]:
print("N_networktypes: ")
print(data['N_networktypes'])
print("N_id : ")
print(data['N_id'])
print("N_responses : ")
print(data['N_responses'])
print("N_periods : ")
print(data['N_periods'])
print("N_periods : ")
print(data['N_periods'])
print("N_individual_predictors : " )
print(data['N_individual_predictors'])
print("N_dyadic_predictors : ")
print(data['N_dyadic_predictors'])
print("flows : ")
print(data['flows'])
data['individual_predictors'] = jnp.array(data['individual_predictors']['Mass'])
print("individual_predictors : ")
print(data['individual_predictors'])
print("N_block_predictors : ")
print(data['N_block_predictors'])
print("N_groups_per_block_type : ")
print(data['N_groups_per_block_type'])
print("outcome_mode : ")
print(data['outcome_mode'] )
data['block_predictors'] = jnp.array(data['block_predictors'])
print("block_predictors : ")
print(data['block_predictors'] )
print("outcome_mode : ")
print(data['outcome_mode'] )

N_networktypes: 
1
N_id : 
100
N_responses : 
1
N_periods : 
0
N_periods : 
0
N_individual_predictors : 
1
N_dyadic_predictors : 
2
flows : 
0
individual_predictors : 
[0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1
 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 1 0
 0 0 0 1 0 1 1 1 0 0 0 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1]
N_block_predictors : 
0
N_groups_per_block_type : 
1
outcome_mode : 
2
block_predictors : 
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
outcome_mode : 
2


In [ ]:
Kinship = jnp.array(data['dyadic_predictors']['Kinship']).reshape(data['N_id'],data['N_id'])
Kinship
Dominant = jnp.array(data['dyadic_predictors']['Dominant']).reshape(data['N_id'],data['N_id'])
Dominant
exposure = jnp.array(data['exposure']).reshape(data['N_id'],data['N_id'])
exposure

Array([[14, 16, 16, ...,  8, 19, 21],
       [13, 11, 16, ..., 15, 21, 26],
       [14, 17, 13, ..., 17, 18, 14],
       ...,
       [22, 18, 15, ..., 14,  6, 13],
       [15, 17, 11, ...,  6, 15, 20],
       [10, 14, 19, ..., 16, 14, 15]], dtype=int32)

In [ ]:
import jax.random as jaxr
@jit
def batch_matrix_vector_multiplication(A, v):
    """
    Perform matrix-vector multiplication for each row of the array v.

    Parameters:
    A (jax.numpy.ndarray): A 2x2 matrix.
    v (jax.numpy.ndarray): An array of shape (n, 2) where each row is a 2-vector.

    Returns:
    jax.numpy.ndarray: An array of shape (n, 2) where each row is the result of the matrix-vector multiplication.
    """
    # Define a function that performs the matrix-vector multiplication
    def matvec(A, v):
        return jnp.dot(A, v)
    
    # Vectorize the function using jax.vmap
    vmap_matvec = jax.vmap(lambda v: matvec(A, v))
    
    # Apply the vectorized function to the array of vectors
    result = vmap_matvec(v)
    
    return result

@jit
def transform_matrix(X):
    Y = []
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            if i != j:
                Y.append(jnp.array([X[i, j], X[j, i]]))
    return jnp.concatenate(Y).reshape(-1, 2)

# vec_to_mat ------------------------------------------------------------------
@jit
def vec_to_mat(vec):
    # Repeat the array to fill a 10x10 matrix
    m = jnp.transpose(jnp.tile(vec, (vec.shape[0] , 1)))
    m = jnp.where(jnp.eye(m.shape[0], dtype=bool), 0, m)
    return m

## Example ------------------------------------------------------------------
print("vec_to_mat: ")
v = jnp.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(vec_to_mat(v))


# Triangles ------------------------------------------------------------------
def upper_tri(array, diag=1):
    """Extracts the upper triangle elements of a 2D JAX array.

    Args:
        array (2D array): A JAX 2D array.
        diag (int): Integer indicating if diagonal must be kept or not.
                    diag=1 excludes the diagonal, diag=0 includes it.
    """
    upper_triangle_indices = jnp.triu_indices(array.shape[0], k=diag)
    upper_triangle_elements = array[upper_triangle_indices]
    return upper_triangle_elements
# JIT compile the function with static_argnums
get_upper_tri = jit(upper_tri, static_argnums=(1,))


def lower_tri(array, diag=-1):
    """Extracts the lower triangle elements of a 2D JAX array.

    Args:
        array (2D array): A JAX 2D array.
        diag (int): Integer indicating if diagonal must be kept or not.
                    diag=0 includes the diagonal, diag=-1 excludes it.
    """
    lower_triangle_indices = jnp.tril_indices(array.shape[0], k=diag)
    lower_triangle_elements = array[lower_triangle_indices]
    return lower_triangle_elements
# JIT compile the function with static_argnums
get_lower_tri = jit(lower_tri, static_argnums=(1,))


def get_tri(array, type='upper', diag=0):
    """Extracts the upper, lower, or both triangle elements of a 2D JAX array.

    Args:
        array (2D array): A JAX 2D array.
        type (str): A string indicating which part of the triangle to extract.
                    It can be 'upper', 'lower', or 'both'.
        diag (int): Integer indicating if diagonal must be kept or not.
                    diag=1 excludes the diagonal, diag=0 includes it.

    Returns:
        If argument type is 'upper', 'lower', it return a 1D JAX array containing the requested triangle elements.
        If argument type is 'both', it return a 2D JAX array containing the the first column the lower triangle and in the second ecolumn the upper triangle
    """
    if type == 'upper':
        upper_triangle_indices = jnp.triu_indices(array.shape[0], k=diag)
        triangle_elements = array[upper_triangle_indices]
    elif type == 'lower':
        lower_triangle_indices = jnp.tril_indices(array.shape[0], k=-diag)
        triangle_elements = array[lower_triangle_indices]
    elif type == 'both':
        upper_triangle_indices = jnp.triu_indices(array.shape[0], k=diag)
        lower_triangle_indices = jnp.tril_indices(array.shape[0], k=-diag)
        upper_triangle_elements = array[upper_triangle_indices]
        lower_triangle_elements = array[lower_triangle_indices]
        triangle_elements = jnp.stack((lower_triangle_elements, upper_triangle_elements), axis = 1)
    else:
        raise ValueError("type must be 'upper', 'lower', or 'both'")

    return triangle_elements

## Example ------------------------------------------------------------------
array = jnp.array([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9]])

print("get_upper_tri: ")
result1 = get_upper_tri(array, 1)
print(result1)

print("get_lower_tri: ")
result2 = get_lower_tri(array, -1)  # Change diag to -1 to exclude the diagonal
print(result2)

# JIT compile the function with static_argnums
get_triangle = jit(get_tri, static_argnames=('type', 'diag'))

print("On function for upper,  lower and both triangles: ")
# Test the function
result_upper = get_triangle(array, 'upper', 1)
print("Upper triangle elements (excluding diagonal):", result_upper)

result_lower = get_triangle(array, 'lower', 1)
print("Lower triangle elements (excluding diagonal):", result_lower)

result_both = get_triangle(array, 'both', 1)
print("Both triangle elements (excluding diagonal):", result_both)


# Dot product betwee matrix and 2x2 cov mat-----------------------------
# Seed for reproducibility
seed = 0
key = jaxr.PRNGKey(seed)

# Define shapes
shape_M = (10, 10)
shape_diag = (2, 2)

# Generate random matrices
sr_raw_M = jaxr.normal(key, shape_M)  # Random matrix with shape (10, 10)
diag = jaxr.normal(key, shape_diag)   # Random matrix with shape (2, 2)

# Initialize the result matrix with zeros
result = jnp.zeros(shape_M)

# Compute the result matrix
@jit
def dot_mat_cov(mat, cov):
    """dot productbetween [sr_raw_M[i,j],sr_raw_M[j,i]] and diag for each combinations of i and j. where M is a 2x2 matrix of shape 10x10 and diag a 2x2x matrix of 2x2. The return object should be a matrix of shape equal to M.

    Args:
        mat (2d jax array): _description_
        cov (2d jax array): _description_
    """
    def compute_ij(i, j):
        # Create the 2x2 tensor from sr_raw_M for indices (i, j)
        tensor_2 = jnp.array([[mat[i, j], mat[j, i]]])
        
        # Perform the dot product with cov
        return jnp.einsum('ij,ij->',tensor_2 , cov)
    
    # Create a result matrix with the same shape as sr_raw_M
    result_matrix = jnp.zeros_like(mat)
    
    # Vectorized computation using broadcasting
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            result_matrix = result_matrix.at[i, j].set(compute_ij(i, j))
    
    return result_matrix

# Compute the result matrix
result_matrix = dot_mat_cov(sr_raw_M, diag)

# Display the resulting matrix
print("Dot product betwee matrix and 2x2 cov mat")
print(result_matrix)


# sr_to_dr_shape-----------------------------
@jit
def sr_to_dr_shape(sr):      
    # Extract columns
    sr1 = array[:, 0] # i to j value
    sr2 = array[:, 1] # j to i value

    # Create a grid of indices
    N = sr.shape[0]
    i_indices, j_indices = jnp.tril_indices(N, -1)  # Indices for upper triangle, excluding diagonal

    # Generate combinations
    return jnp.stack([sr1[i_indices], sr2[j_indices]], axis=-1)


array = jnp.array([
    [1, 2],
    [3, 4],
    [5, 6],
    [7, 8]
])
print("sr_to_dr_shape:")
print(sr_to_dr_shape(array))



vec_to_mat: 
[[ 0  1  1  1  1  1  1  1  1  1]
 [ 2  0  2  2  2  2  2  2  2  2]
 [ 3  3  0  3  3  3  3  3  3  3]
 [ 4  4  4  0  4  4  4  4  4  4]
 [ 5  5  5  5  0  5  5  5  5  5]
 [ 6  6  6  6  6  0  6  6  6  6]
 [ 7  7  7  7  7  7  0  7  7  7]
 [ 8  8  8  8  8  8  8  0  8  8]
 [ 9  9  9  9  9  9  9  9  0  9]
 [10 10 10 10 10 10 10 10 10  0]]
get_upper_tri: 
[2 3 6]
get_lower_tri: 
[4 7 8]
On function for upper,  lower and both triangles: 
Upper triangle elements (excluding diagonal): [2 3 6]
Lower triangle elements (excluding diagonal): [4 7 8]
Both triangle elements (excluding diagonal): [[4 2]
 [7 3]
 [8 6]]
Dot product betwee matrix and 2x2 cov mat
[[-1.7161117   3.8507886   2.6675742  -1.2665142  -2.5273845   1.5500902
  -0.56202716 -0.19732404 -1.3603567   0.8664815 ]
 [-4.0113864  -1.0849247   0.08289722  4.2800307  -0.29872686  1.4720303
  -3.766257   -2.42005    -4.7452354   2.517879  ]
 [-2.399936    2.1465735   0.24681942  1.4653203  -0.04389387 -1.331646
  -0.42534143 -3.004

In [ ]:
# Matrices to edgelist
result_dom = get_triangle(Dominant, 'both', 1)
result_dom.shape
result_kin = get_triangle(Kinship, 'both', 1)
result_kin
data['outcomes'] = jnp.array(data['outcomes']).reshape(data['N_id'],data['N_id'])
result_outcomes = get_triangle(data['outcomes'], 'both', 1)
result_outcomes.shape

N_id = data['individual_predictors'].shape[0]
print((N_id*(N_id-1))/2)
focal_individual_predictors = data['individual_predictors']
target_individual_predictors = data['individual_predictors']


4950.0


In [ ]:

# Building model and sampling it ------------------
def model(N_id, result_outcomes, result_kin, focal_individual_predictors, target_individual_predictors):
    # SR term
    focal_effects =  numpyro.sample('focal_effects', numpyro.distributions.Normal(0,1).expand([1]))
    target_effects =  numpyro.sample('target_effects', numpyro.distributions.Normal(0,1).expand([1]))
    sr_raw = numpyro.sample('sr_raw', numpyro.distributions.Normal(0,1).expand([N_id, 2]))
    sr_sigma = numpyro.sample('sr_sigma', numpyro.distributions.Exponential(1).expand([2]))
    sr_L = numpyro.sample("sr_L", numpyro.distributions.LKJ(2, 4))

    ## SR term computation
    X = batch_matrix_vector_multiplication(sr_L @ jnp.diag(sr_sigma), sr_raw)
    terms = jnp.stack([focal_effects * focal_individual_predictors, target_effects * target_individual_predictors], axis = 1)
    sr = terms + X
    sr = sr_to_dr_shape(sr) # To compute the log probability, we need to reshape `sr` and `outcomes` to fit `dr`. A matrix approach should be possible or convert `sr` and `dr` to matrices.
    sr = sr.astype(jnp.float32)

    # Dyadic term
    dyad_effects = numpyro.sample('dyad_effects', numpyro.distributions.Normal(0,1).expand([2]))
    terms2 = dyad_effects * result_kin
    dr_raw = numpyro.sample('dr_raw', numpyro.distributions.Normal(0,1).expand([terms2.shape[0],2]))
    dr_sigma = numpyro.sample('dr_sigma', numpyro.distributions.Exponential(1).expand([1]))
    dr_L = numpyro.sample("dr_L", numpyro.distributions.LKJ(2, 4))
    X2 = batch_matrix_vector_multiplication(dr_L, dr_raw) * jnp.repeat(dr_sigma,2)
    dr = X2 + terms2

    numpyro.sample('Y', numpyro.distributions.Poisson(sr + dr), obs=result_outcomes)

dat = dict(
    N_id = N_id,
    result_outcomes = result_outcomes,
    result_kin = result_kin, 
    focal_individual_predictors = focal_individual_predictors,
    target_individual_predictors = target_individual_predictors
)
m = MCMC(NUTS(model, init_strategy = numpyro.infer.init_to_median()), num_warmup=500, num_samples=500, num_chains=1)
m.run(random.PRNGKey(0), extra_fields=["diverging"], **dat)

sample: 100%|██████████| 1000/1000 [00:22<00:00, 45.45it/s, 8 steps of size 2.65e-02. acc. prob=0.79] 


In [ ]:
data = az. from_numpyro(m)
df = az.summary(data)

arviz - WARNING - Shape validation failed: input_shape: (1, 500), minimum_shape: (chains=2, draws=4)


In [32]:
df.index

Index(['dr_L[0, 0]', 'dr_L[0, 1]', 'dr_L[1, 0]', 'dr_L[1, 1]', 'dr_raw[0, 0]',
       'dr_raw[0, 1]', 'dr_raw[1, 0]', 'dr_raw[1, 1]', 'dr_raw[2, 0]',
       'dr_raw[2, 1]',
       ...
       'sr_raw[96, 1]', 'sr_raw[97, 0]', 'sr_raw[97, 1]', 'sr_raw[98, 0]',
       'sr_raw[98, 1]', 'sr_raw[99, 0]', 'sr_raw[99, 1]', 'sr_sigma[0]',
       'sr_sigma[1]', 'target_effects[0]'],
      dtype='object', length=10115)

In [18]:
for i in df.index:
    print(i)

dr_L[0, 0]
dr_L[0, 1]
dr_L[1, 0]
dr_L[1, 1]
dr_raw[0, 0]
dr_raw[0, 1]
dr_raw[1, 0]
dr_raw[1, 1]
dr_raw[2, 0]
dr_raw[2, 1]
dr_raw[3, 0]
dr_raw[3, 1]
dr_raw[4, 0]
dr_raw[4, 1]
dr_raw[5, 0]
dr_raw[5, 1]
dr_raw[6, 0]
dr_raw[6, 1]
dr_raw[7, 0]
dr_raw[7, 1]
dr_raw[8, 0]
dr_raw[8, 1]
dr_raw[9, 0]
dr_raw[9, 1]
dr_raw[10, 0]
dr_raw[10, 1]
dr_raw[11, 0]
dr_raw[11, 1]
dr_raw[12, 0]
dr_raw[12, 1]
dr_raw[13, 0]
dr_raw[13, 1]
dr_raw[14, 0]
dr_raw[14, 1]
dr_raw[15, 0]
dr_raw[15, 1]
dr_raw[16, 0]
dr_raw[16, 1]
dr_raw[17, 0]
dr_raw[17, 1]
dr_raw[18, 0]
dr_raw[18, 1]
dr_raw[19, 0]
dr_raw[19, 1]
dr_raw[20, 0]
dr_raw[20, 1]
dr_raw[21, 0]
dr_raw[21, 1]
dr_raw[22, 0]
dr_raw[22, 1]
dr_raw[23, 0]
dr_raw[23, 1]
dr_raw[24, 0]
dr_raw[24, 1]
dr_raw[25, 0]
dr_raw[25, 1]
dr_raw[26, 0]
dr_raw[26, 1]
dr_raw[27, 0]
dr_raw[27, 1]
dr_raw[28, 0]
dr_raw[28, 1]
dr_raw[29, 0]
dr_raw[29, 1]
dr_raw[30, 0]
dr_raw[30, 1]
dr_raw[31, 0]
dr_raw[31, 1]
dr_raw[32, 0]
dr_raw[32, 1]
dr_raw[33, 0]
dr_raw[33, 1]
dr_raw[34, 0]
dr_raw[34,

In [ ]:
df.loc[['sr_sigma[0]', 'sr_sigma[1]']]

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
sr_sigma[0],0.515,0.021,0.487,0.553,0.016,0.013,3.0,18.0,NaN
sr_sigma[1],0.735,0.014,0.716,0.760,0.012,0.011,2.0,23.0,NaN


In [ ]:
df.loc[["focal_effects[0]", "target_effects[0]"]]

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
focal_effects[0],-0.176,0.055,-0.293,-0.088,0.039,0.033,2.0,11.0,NaN
target_effects[0],0.153,0.015,0.129,0.178,0.014,0.015,1.0,22.0,NaN


In [29]:
pip install pystan

  Using cached pystan-3.10.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached aiohttp-3.10.3-cp312-cp312-win_amd64.whl.metadata (7.8 kB)
  Using cached clikit-0.6.2-py2.py3-none-any.whl.metadata (1.6 kB)
INFO: pip is looking at multiple versions of pystan to determine which version is compatible with other requirements. This could take a while.
  Using cached pystan-3.9.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached pystan-3.9.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached pystan-3.8.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached pystan-3.7.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached pystan-3.6.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached pystan-3.5.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached pystan-3.4.0-py3-none-any.whl.metadata (3.7 kB)
INFO: pip is still looking at multiple versions of pystan to determine which version is compatible with other requirements. This could take a while.
  Using cached pystan-3.3.0-py3-none-any.whl.metadata (3.6 kB)

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [4 lines of output]
      C:\Users\SSosa\AppData\Local\Temp\pip-install-q39hrkk_\pystan_b2f193c4536f4bf193c6b26fcb9c7eea\setup.py:61: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
        self.version = node.value.s
      Cython>=0.22 and NumPy are required.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pystan
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pystan)


In [31]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()

stan_code = """
data{
    int N_networktypes;                                               
    int N_id;                                                                                                            
    int N_responses;        

    array[3] int N_params;                                          
                                             
    array[N_id,N_id,N_responses] int outcomes;  
    array[N_id,N_id,N_responses] int exposure;                              

    matrix[N_id, N_params[1]] focal_set;
    matrix[N_id, N_params[2]] target_set;

    array[N_id, N_id, N_params[3]] real dyad_set;

    matrix [22, 2] priors;
    
    int export_network;
    int outcome_mode;                           
}

transformed data{
 matrix[N_id, N_params[1]-1] focal_individual_predictors; 
 matrix[N_id, N_params[2]-1] target_individual_predictors; 

 array[N_id, N_id, N_params[3]-1] real dyad_individual_predictors; 

//# Make pruned data
  
  if(N_params[1]>1){
  for(i in 2:N_params[1]){
  focal_individual_predictors[ , i-1] = focal_set[,i];  
   }}

  if(N_params[2]>1){
  for(i in 2:N_params[2]){
  target_individual_predictors[ , i-1] = target_set[,i];  
   }}

  if(N_params[3]>1){
  for(i in 2:N_params[3]){
  dyad_individual_predictors[ , , i-1] = dyad_set[,,i];  
   }}
}

parameters{
    //########################################################### Latent Netowrk
    matrix[1,1] B;

    vector<lower=0>[2] sr_sigma;  //# Variation of sender-receiver effects
    cholesky_factor_corr[2] sr_L;
    array[N_id] vector[2] sr_raw;

    real<lower=0> dr_sigma;     //# Variation of dyadic effects
    cholesky_factor_corr[2] dr_L;
    matrix[N_id, N_id] dr_raw;

    //# Effects of covariate
    vector[N_params[1]-1] focal_effects;
    vector[N_params[2]-1] target_effects;
    vector[N_params[3]-1] dyad_effects;  
}

model{
  array[N_id] vector[2] sr;
  matrix[N_id, N_id] dr;

  vector[2] scrap;

    //# Priors on effects of covariates
     focal_effects ~ normal(priors[12,1], priors[12,2]);
     target_effects ~ normal(priors[13,1], priors[13,2]);
     dyad_effects ~ normal(priors[14,1], priors[14,2]);

    //# Sender-receiver priors for social relations model
    for(i in 1:N_id)
    sr_raw[i] ~ normal(0,1);

    sr_sigma ~ exponential(priors[15,1]);    
    sr_L ~ lkj_corr_cholesky(priors[17,1]);

    for(i in 1:N_id){
     vector[2] sr_terms;

     sr_terms[1] = dot_product(focal_effects,  to_vector(focal_individual_predictors[i]));
     sr_terms[2] = dot_product(target_effects,  to_vector(target_individual_predictors[i]));  

     sr[i] = diag_pre_multiply(sr_sigma, sr_L) * sr_raw[i] + sr_terms;
     }

    //# Dyadic priors for social relations model
    to_vector(dr_raw) ~ normal(0,1);
    dr_sigma ~ exponential(priors[16,1]);
    dr_L ~ lkj_corr_cholesky(priors[18,1]);

    for(i in 1:(N_id-1)){
    for(j in (i+1):N_id){
     scrap[1] = dr_raw[i,j];
     scrap[2] = dr_raw[j,i];
     scrap = rep_vector(dr_sigma, 2) .* (dr_L*scrap);
     dr[i,j] = scrap[1] + dot_product(dyad_effects,  to_vector(dyad_individual_predictors[i, j, ]));
     dr[j,i] = scrap[2] + dot_product(dyad_effects,  to_vector(dyad_individual_predictors[j, i, ]));
     }
     }

    for(i in 1:N_id){
     dr[i,i] = -99; //# ignore this :)
    }

    //# priors for 
    B[1,1] ~ normal(logit(priors[10,1]/sqrt(N_id)), priors[10,2]);


    //# likelihood
    for ( i in 1:N_id ) {
     for ( j in 1:N_id ) {
       if ( i != j ) {

      if(outcome_mode==1){
      outcomes[i,j,1] ~ bernoulli_logit(B[1,1] + sr[i,1] + sr[j,2] + dr[i,j]);  //# Then model the outcomes
       }
      if(outcome_mode==2){
      outcomes[i,j,1] ~ binomial_logit(exposure[i,j,1], B[1,1] + sr[i,1] + sr[j,2] + dr[i,j]);  //# Then model the outcomes
       }
      if(outcome_mode==3){
      outcomes[i,j,1] ~ poisson_log(B[1,1] + sr[i,1] + sr[j,2] + dr[i,j]);  //# Then model the outcomes
       }

       }
      }
     }


 }


generated quantities{
    //# compute posterior prob of each network tie
    matrix[N_id*export_network, N_id*export_network] p;
    array[N_id*export_network] vector[2*export_network] sr;
    matrix[N_id*export_network, N_id*export_network] dr;
 
    if(export_network==1){                
     vector[2] terms;
     int tie;
     vector[2] scrap;
            
    for(i in 1:N_id){
     vector[2] sr_terms;

     sr_terms[1] = dot_product(focal_effects,  to_vector(focal_individual_predictors[i]));
     sr_terms[2] = dot_product(target_effects,  to_vector(target_individual_predictors[i]));  

     sr[i] = diag_pre_multiply(sr_sigma, sr_L) * sr_raw[i] + sr_terms;
     }

    for(i in 1:(N_id-1)){
    for(j in (i+1):N_id){
     scrap[1] = dr_raw[i,j];
     scrap[2] = dr_raw[j,i];
     scrap = rep_vector(dr_sigma, 2) .* (dr_L*scrap);
     dr[i,j] = scrap[1] + dot_product(dyad_effects,  to_vector(dyad_individual_predictors[i, j, ])) + B[1, 1];
     dr[j,i] = scrap[2] + dot_product(dyad_effects,  to_vector(dyad_individual_predictors[j, i, ])) + B[1, 1];
    }
    }

    for(i in 1:N_id){
     dr[i,i] = -99; //# ignore this :)
    }


    for ( i in 1:N_id ) {
        for ( j in 1:N_id ) {
            if ( i != j ) {
      // consider each possible state of true tie and compute prob of data
      if(outcome_mode==1){
       p[i,j] = inv_logit( sr[i,1] + sr[j,2] + dr[i,j]);
       }
      if(outcome_mode==2){
       p[i,j] = inv_logit( sr[i,1] + sr[j,2] + dr[i,j]);
       }
      if(outcome_mode==3){
       p[i,j] = exp(sr[i,1] + sr[j,2] + dr[i,j]);  
       }
            }
        }//j
    }//i

  for ( i in 1:N_id ) {
   p[i,i] = 0; 
   }
 }
}
"""
data = {
    'blooms_std' : d["blooms_std"].values,
    "water_cent": d["water_cent"].values.astype(int),
    "shade_cent": d["shade_cent"].values.astype(int),
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=500, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

ModuleNotFoundError: No module named 'pystan'

In [24]:
pip install Numpy

Note: you may need to restart the kernel to use updated packages.


# Dirichlet Multinomial with centered random factors

In [28]:
import seaborn as sns
import numpy as np
from jax import random
from jax.nn import softmax
import jax.numpy as jnp
import numpyro as numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, Predictive

###############################################################################
############ SIMULATING MULTINOMIAL DATA WITH SOFTMAX LINK FUNCTION ###########
def mysoftmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x, axis=0)

K = 3
N = 100
N_obs = 2
sigma_random = 0.6


########################################################
################### Fixed effect Sim ###################
#a = np.random.normal(0, 1, K)
a = np.array([3,1,1]) # Forcing a values


# Factors--------------------------
NY = 4
NV = 8

Y2 = np.full((NV, NY), np.nan) 
means = np.random.normal(0, 1, NY)
offsets = np.random.normal(0, 1, NV)
for i in range(NV):
  for k in range(NY):
    Y2[i,k] = means[k] + offsets[i]

    
b_individual = np.random.normal(0, 1, (N, K))
mu = b_individual + a


# Declare an empty Matrix to fill with data
Y = np.empty((N * N_obs, K))

# Declare an empty vector to fill with IDs
id = []

# Loop over each individual
for i in range(N):
    # Simulate N_obs draws from the multinomial
    Y[i*N_obs:(i+1)*N_obs, :] = np.apply_along_axis(lambda x: np.random.multinomial(100, mysoftmax(x)), 0, mu[i])
    # Assign ID vector
    id += [i] * N_obs


N = N*N_obs
K = K
ni = N
y = jnp.array(Y, dtype=jnp.int32).reshape(N, K)
i_ID = jnp.array(id)

dat = dict(
    K = K,
    ni = ni,
    y = y,
    i_ID = i_ID
)

## Latent variable model

In [29]:
init_key, sample_key = random.split(random.PRNGKey(0))
init_key = jnp.array(init_key)
init_key

Array([4146024105,  967050713], dtype=uint32)

In [30]:
import pandas as pd 
import math
import os
import seaborn as sns
import arviz as az
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import Image, set_matplotlib_formats
from matplotlib.patches import Ellipse, transforms
import jax
import jax.numpy as jnp
from jax import random, vmap
from jax.scipy.special import expit

import numpy as onp
import numpyro as numpyro

import numpyro.distributions as dist
from numpyro.diagnostics import effective_sample_size, print_summary
from numpyro.infer import MCMC, NUTS
numpyro.set_platform("cpu")
numpyro.set_host_device_count(30)


# Simulation ---------------
NY = 4
NV = 8

Y2 = np.full((NV, NY), np.nan) 
means = np.random.normal(0, 1, NY)
offsets = np.random.normal(0, 1, NV)
for i in range(NV):
  for k in range(NY):
    Y2[i,k] = means[k] + offsets[i]

b_individual = np.random.normal(0, 1, (N, K))
mu = b_individual + a

Y2 = jnp.array(Y2)



ModuleNotFoundError: No module named 'networkx'

In [ ]:

# Building model and sampling it ------------------
def model(NY, NV, Y2):
    means =  numpyro.sample('means', numpyro.distributions.Normal(0,1).expand([NY]))
    offset =  numpyro.sample('offset', numpyro.distributions.Normal(0,1).expand([NV,1]))
    sigma =  numpyro.sample('sigma', numpyro.distributions.Exponential(1).expand([NY])) 
    tmp = jnp.tile(means, (NV, 1)).reshape(NV,NY)  
    mu_l = tmp + offset 
    numpyro.sample('Y2', numpyro.distributions.Normal(mu_l, jnp.tile(sigma, [NV, 1])), obs=Y2)

dat = dict(
    NY = NY,
    NV = NV,
    Y2 = Y2
)
m = MCMC(NUTS(model, init_strategy = numpyro.infer.init_to_median()), num_warmup=1000, num_samples=1000, num_chains=1)
m.run(random.PRNGKey(0), extra_fields=["diverging"], **dat)

In [15]:
import arviz as az
data = az.from_numpyro(m)
az.summary(data)

arviz - WARNING - Shape validation failed: input_shape: (1, 1000), minimum_shape: (chains=2, draws=4)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
means[0],-1.601,0.0,-1.601,-1.601,0.0,0.0,1000.0,1000.0,NaN
means[1],-0.331,0.0,-0.331,-0.331,0.0,0.0,1000.0,1000.0,NaN
means[2],0.154,0.0,0.154,0.154,0.0,0.0,1000.0,1000.0,NaN
means[3],-0.416,0.0,-0.416,-0.416,0.0,0.0,1000.0,1000.0,NaN
"offset[0, 0]",1.167,0.0,1.167,1.167,0.0,0.0,1000.0,1000.0,NaN
"offset[1, 0]",-0.976,0.0,-0.976,-0.976,0.0,0.0,1000.0,1000.0,NaN
"offset[2, 0]",0.520,0.0,0.520,0.520,0.0,0.0,1000.0,1000.0,NaN
"offset[3, 0]",-0.537,0.0,-0.537,-0.537,0.0,0.0,1000.0,1000.0,NaN
"offset[4, 0]",-0.941,0.0,-0.941,-0.941,0.0,0.0,1000.0,1000.0,NaN
"offset[5, 0]",0.945,0.0,0.945,0.945,0.0,0.0,1000.0,1000.0,NaN


In [16]:
means

array([-1.310413  , -0.0403644 ,  0.44440847, -0.12578016])

## Random centered effects

In [7]:
@jit
def random_centered(sigma, cor_mat, offset_mat):
    """Generate the centered matrix of random factors 

    Args:
        sigma (vector): Prior, vector of length N
        cor_mat (2D array): correlation matrix, cholesky_factor_corr of dim N, N
        offset_mat (2D array): matrix of offsets, matrix of dim N*k

    Returns:
        _type_: 2D array
    """
    return jnp.dot(diag_pre_multiply(sigma, cor_mat), offset_mat)

In [17]:
def model(K, ni, y, i_ID):
    a = normal('a', [K], 0,1)
    Sigma_individual = exponential('Sigma_individual', [ni], 1 )
    L_individual = lkjcholesky('L_individual', [], ni, 1) # Implies a uniform distribution over correlation matrices
    z_individual = normal('z_individual', [ni,K], 0, 1)
    alpha = random_centered(Sigma_individual, L_individual, z_individual)
    lk = jnp.exp(a + alpha[i_ID])
    sample("y", DirichletMultinomial(lk, int(100)), obs=y)

m = bi()
m.data = dat
m.run(model, init_strategy = numpyro.infer.init_to_median(), 
      num_warmup=500, num_samples=500, num_chains=1)


jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [01:47<00:00,  9.27it/s, 35 steps of size 3.91e-07. acc. prob=0.80] 


In [15]:
print('Simulated:')
print(jax.nn.softmax(jnp.array(a))) 
print('Numpypro estimation:')
print(jax.nn.softmax(jnp.mean(jnp.array(m.trace['posterior']['a'][0]), axis = 0)))

Simulated:
[0.786986   0.10650697 0.10650697]
Numpypro estimation:
[0.7328625  0.13674371 0.13039377]


In [ ]:
def model(K, ni, y, i_ID):
    a = normal('a', [K], 0,1)
    Sigma_individual = exponential('Sigma_individual', [ni], 1 )
    L_individual = lkjcholesky('L_individual', [], ni, 1) # Implies a uniform distribution over correlation matrices
    z_individual = normal('z_individual', [ni,K], 0, 1)
    alpha = random_centered2(Sigma_individual, L_individual, z_individual)
    lk = jnp.exp(a + alpha[i_ID])
    sample("y", DirichletMultinomial(lk, int(100)), obs=y)

m = bi()
m.data = dat
m.run(model, init_strategy = numpyro.infer.init_to_median(), 
      num_warmup=500, num_samples=500, num_chains=1)

In [49]:
print('Simulated:')
print(jax.nn.softmax(jnp.array(a))) 
print('Numpypro estimation:')
print(jax.nn.softmax(jnp.mean(jnp.array(m.trace['posterior']['a'][0]), axis = 0)))

Simulated:
[0.786986   0.10650697 0.10650697]
Numpypro estimation:
[0.66280484 0.1774538  0.15974137]


In [ ]:
def model(K, ni, y, i_ID):
    a = normal('a', [K], 0,1)
    Sigma_individual = exponential('Sigma_individual', [ni], 1 )
    L_individual = lkjcholesky('L_individual', [], ni, 1) # Implies a uniform distribution over correlation matrices
    print(L_individual.shape)
    z_individual = normal('z_individual', [ni,K], 0, 1)
    alpha = ((Sigma_individual[..., None] * L_individual) @ z_individual)
    print(alpha.shape)
    lk = jnp.exp(a + alpha[i_ID])
    sample("y", DirichletMultinomial(lk, int(100)), obs=y)

m = bi()
m.data = dat
m.run(model, init_strategy = numpyro.infer.init_to_median(), 
      num_warmup=500, num_samples=500, num_chains=1, chain_method='vectorized')

In [56]:
print('Simulated:')
print(jax.nn.softmax(jnp.array(a))) 
print('Numpypro estimation:')
print(jax.nn.softmax(jnp.mean(jnp.array(m.trace['posterior']['a'][0]), axis = 0)))

Simulated:
[0.786986   0.10650697 0.10650697]
Numpypro estimation:


AttributeError: 'bi' object has no attribute 'trace'

In [56]:
###############################################################################
#################################### Pustan Model  #############################
import time as tm
import stan
import nest_asyncio
import numpy as np
tmp = dat
tmp['y'] = np.array(tmp['y'])
tmp['i_ID'] = np.array(tmp['i_ID']+1)
tmp['ni'] = tmp['ni']
tmp['K'] = tmp['K']
tmp['N'] = int(N)

nest_asyncio.apply()
stan_code = """ 
data {
    int<lower=0>  N;             // number of observations
    int<lower=0>  K;             // number of occupations
    int ni;                     // NUmber of Unique Individauls
    array[N, K] int y;           // array of observed occupation indicators
    array[N]int<lower=0>  i_ID;     // village indicator for each individual
}
parameters {
    vector[K] a;                    // intercept for each occupation
    matrix[ni, K]  z_individual;    // raw random effect for household 
    cholesky_factor_corr[ni] L_individual; // Cholesky factor for 
    vector<lower=0>[ni] Sigma_individual;

}
transformed parameters{
    matrix[ni, K] b_individual;
    b_individual = diag_pre_multiply(Sigma_individual, L_individual) * z_individual;
}
model{
    array[N] vector[K] p;
    matrix[N, K] random_effects;
    to_vector(a) ~ normal(0, 1);
    L_individual ~   lkj_corr_cholesky(2);
    Sigma_individual ~ exponential(1);
    to_vector(z_individual) ~ normal(0, 1);
    // Likelihood for
    for (k in 1:K) {
        for (i in 1:N) {
          random_effects[i, k] = b_individual[i_ID[i], k];
          p[i,k] =  a[k] + random_effects[i, k];
      }
    }
    for (i in 1:(N)) {
        y[i,] ~ dirichlet_multinomial(exp(p[i,]));
    }
}
"""

start = tm.time()
stan_model = stan.build(stan_code, data = tmp)
fit = stan_model.sample(num_chains=1, num_samples=500, num_warmup = 500, init = [{'L_individual': np.zeros((tmp['ni'], tmp['ni']))}])
end = tm.time()    
#df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: found in cache, done.Sampling:   0%/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

Sampling:   0% (1/1000)
Sampling:  10% (100/1000)
Sampling:  20% (200/1000)
Sampling:  30% (300/1000)
Sampling:  40% (400/1000)
Sampling:  50% (500/1000)
Sampling:  50% (501/1000)
Sampling:  60% (600/1000)
Sampling:  70% (700/1000)
Sampling:  80% (800/1000)
Sampling:  90% (900/1000)
Sampling: 100% (1000/1000)
Sampling: 100% (1000/1000), done.
Messages received during sampling:
  Gradient evaluation took 0.006054 seconds
  1000 transitions using 10 leapfrog steps per transition would take 60.54 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_cholesky_lpdf: Random variable[6] is 0

Pystan took: 432.2593 seconds


In [110]:
print('Simulated:')
print(jax.nn.softmax(jnp.array(np.array([3,1,1])))) 
print('Estimation Multinomial:')
post = m.sampler.get_samples()
print(jax.nn.softmax(jnp.mean(post['a'], axis = 0)))
print('Estimation DirichletMultinomial:')
#post = m2.sampler.get_samples()
#print(jax.nn.softmax(jnp.mean(post['a'], axis = 0)))
df = fit.to_frame()
print('Pytstan estimation')
print(jax.nn.softmax(jnp.array([df['a.1'].mean(),df['a.2'].mean(),df['a.3'].mean()])))

Simulated:
[0.786986   0.10650697 0.10650697]
Estimation Multinomial:
[0.79368174 0.11202765 0.0942907 ]
Estimation DirichletMultinomial:
Pytstan estimation
[0.7914235  0.11247264 0.09610377]


## Gaussian process

In [ ]:
# Distance matrix 
L = jnp.arange(1,11)
m = jnp.abs(L[:, None] - L[None, :])

# Kernel
@jit
def sq_exp_gp(m, sq_alpha = .5, sq_rho = .1, delta = 0):
    N = m.shape[0]
    K = jnp.full((N,N),  sq_alpha + delta)
    K = K + sq_alpha * jnp.exp(-sq_rho * m * m )

    mask = jnp.triu(jnp.ones_like(K, dtype=bool))
    cov = jnp.where(mask, K, 0)
    return K, cov

r = sq_exp_gp(m)
Z = Normal(0,1).sample(10)
a = r[1] @ jnp.transpose(r[1]) @ Z

In [52]:
L.shape[0]

10

## Combine latent, random and gaussian

In [ ]:
# latent + random = random + latent[village[i]] with village being village ID
# gaussian simulation 
# 1. non linear function to generate gaussian proces for each parameters
# model 2 = random + latent[village[i]] + gaussian_process
# model 3  = model 2 + interaction effect
#interaction effect = non linear function where input is hhmembers[i]*offsets[vilage[i]] with new coefficients params
# Within model we need to change offset ouput as an integer so we do bxi*hhmembers[i]*offset[v_ID[i]]+ bxIsq*(hhmembers[i]*offset[v_ID[i]])


# Model on real data

In [ ]:
def model(K, ni, y, i_ID):
    #individual 
    Sigma_individual = exponential('Sigma_individual', [ni], 1 )
    L_individual = lkjcholesky('L_individual', [], ni, 50)
    z_individual = normal('z_individual', [ni,K], 0, 1)
    alpha_individual = random_centered2(Sigma_individual, L_individual, z_individual)

    #household 
    Sigma_household = exponential('Sigma_household', [ni], 1 )
    L_household = lkjcholesky('L_household', [], ni, 50)
    z_household = normal('z_household', [ni,K], 0, 1)
    alpha_household = random_centered2(Sigma_household, L_household, z_household)

    #village 
    Sigma_village = exponential('Sigma_village', [ni], 1 )
    L_village = lkjcholesky('L_village', [], ni, 50)
    z_village = normal('z_village', [ni,K], 0, 1)
    alpha_village = random_centered2(Sigma_village, L_village, z_village)

    #LK
    random_factors = alpha_individual[i_ID] + alpha_household[i_ID] + alpha_village[i_ID]
    numpyro.sample("y", dist.DirichletMultinomial(a + random_factors[i_ID], int(12083)), obs=y)

# Test parallelized random centered

In [ ]:
init_key, sample_key = random.split(random.PRNGKey(int(r.randint(0, 10000000))))
init_key = jnp.array(init_key)

Sigma_i = Exponential(1).sample(init_key, [ni,])
L_i= LKJCholesky(ni, 1).sample(init_key, [])
z_i= Normal(0, 1).sample([ni,K])
alpha = random_centered(Sigma_i, L_i, z_i)

# Model to latex

In [57]:
import re
from IPython.display import display, Latex
greek_symbols = {
    'alpha': '\\alpha',
    'beta': '\\beta',
    'gamma': '\\gamma',
    'delta': '\\delta',
    'epsilon': '\\epsilon',
    'zeta': '\\zeta',
    'eta': '\\eta',
    'theta': '\\theta',
    'iota': '\\iota',
    'kappa': '\\kappa',
    'lambda': '\\lambda',
    'mu': '\\mu',
    'nu': '\\nu',
    'xi': '\\xi',
    'omicron': 'o',  # No direct LaTeX symbol for omicron, using "o"
    'pi': '\\pi',
    'rho': '\\rho',
    'sigma': '\\sigma',
    'tau': '\\tau',
    'upsilon': '\\upsilon',
    'phi': '\\phi',
    'chi': '\\chi',
    'psi': '\\psi',
    'omega': '\\omega'
}

def convert_to_greek(var_name):
    # Convert variable name to lowercase for case-insensitive matching
    var_name_lower = var_name.lower()
    # Check if the variable name has a corresponding Greek symbol
    if var_name_lower in greek_symbols:
        return greek_symbols[var_name_lower]
    else:
        return var_name

def extract_latex(command):
    # Define a regular expression pattern to match the desired parts of the command
    pattern = r"(\w+)\s*=\s*(\w+)\([^,]+,\s*[^,]+,\s*(.*)\)"
    match = re.match(pattern, command)
    
    if match:
        var_name = match.group(1)
        func_name = match.group(2)
        params = match.group(3)
        # Convert var_name to Greek symbol if applicable
        var_name_latex = convert_to_greek(var_name)
        # Construct the desired LaTeX text
        latex_text = f"{var_name_latex} = {func_name}({params})"
        return latex_text
    else:
        return None
# Example usage
command = "Sigma_i = exponential('Sigma_individual', [ni], 1)"
latex_text = extract_latex(command)
display(Latex(f'''${latex_text}$'''))

<IPython.core.display.Latex object>